In [21]:
from tensorflow.keras.models import load_model

import os
import multiprocessing
import warnings
import librosa

from tqdm import tqdm
from scipy import stats

import numpy as np
import pandas as pd
import sklearn as skl
import utils

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop, Nadam

import keras

In [23]:
model = Sequential()
model.add(Dense(128, input_dim=207, activation='relu'))

model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               26624     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 2056      
Total params: 61,704
Trainable params: 61,704
Non-trainable params: 0
_________________________________________________________________


In [24]:
model = load_model('models/DNN.h5')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [107]:
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features2(songname):

            features = []           
            
            y, sr = librosa.load(songname, duration=30)
            tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            features.append(tempo)
            features.append(np.sum(beats))
            features.append(np.mean(chroma_stft))
            features.append(np.mean(rmse))
            features.append(np.mean(cent))
            features.append(np.mean(spec_bw))
            features.append(np.mean(rolloff))
            features.append(np.mean(zcr))
            for coefficient in mfcc:
                features.append(np.mean(coefficient))
            

            return features
        
        
def compute_features(filepath):

    features = pd.Series(index=columns(), dtype=np.float32, name=filepath)

    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rms(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}: {}'.format(filepath, repr(e)))

    return features

genre_dict = {
    'Hip-Hop': 0,
    'Pop': 1,
    'Folk': 2,
    'Rock': 3,
    'Experimental': 4,
    'International': 5,
    'Electronic': 6,
    'Instrumental': 7,
}

inv_genre_dict = {v: k for k, v in genre_dict.items()}

genre_names = [inv_genre_dict[i] for i in range(8)]

In [148]:
test = compute_features('songs/club.wav')
test2 = compute_features('songs/rock.wav')

In [149]:
print(test2.head())
test.shape

feature      statistics  number
chroma_cens  kurtosis    01        0.065404
                         02        0.431405
                         03       -0.404023
                         04        0.254728
                         05        0.207687
Name: songs/rock.wav, dtype: float32


(518,)

In [150]:
import pickle
X = test.to_frame().T
X2 = test2.to_frame().T
X = pd.concat([X,X2])

pca = pickle.load(open('pca','rb'))

X = skl.preprocessing.StandardScaler().fit_transform(X)
X = pca.transform(X)

print(inv_genre_dict[model.predict_classes(X)[0]])
model.predict(X)

Electronic


array([[0.08777935, 0.10765458, 0.06566357, 0.13068023, 0.04024485,
        0.15346232, 0.34702435, 0.06749073],
       [0.02657715, 0.21792585, 0.06786897, 0.01259747, 0.20324834,
        0.02076952, 0.16486531, 0.28614733]], dtype=float32)

In [48]:
model.predict(pca.transform(skl.preprocessing.StandardScaler().fit_transform(test2.to_frame().T)))

array([[0.09499289, 0.06599655, 0.14774106, 0.07535625, 0.1112358 ,
        0.11708879, 0.24027342, 0.14731522]], dtype=float32)

In [90]:
tracks = utils.load('fma_metadata/tracks.csv')
genres = utils.load('fma_metadata/genres.csv')

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

small = tracks['set', 'subset'] <= 'small'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

features = utils.load('fma_metadata/features.csv')
X_train = features.loc[small & train]

y_train = tracks.loc[small & train, ('track', 'genre_top')]


In [118]:
'''print(X_train.shape)
X_train = X_train.iloc[0]
print(type(X_train))
print(X_train.shape)
X_train = skl.preprocessing.StandardScaler().fit_transform(X_train.to_frame())

X_train = pca.transform(X_train.T)
predictions = model.predict_classes(X_train)'''
print(X_train.shape)
X_train = skl.preprocessing.StandardScaler().fit_transform(X_train)
print(X_train)
print(X_train.shape)
X_train = pca.transform(X_train)
print(X_train.shape)
predictions = model.predict_classes(X_train)

(6400, 207)
[[ 0.49028565 -0.23754468  1.15523324 ...  0.33788714 -0.1318657
  -0.32881255]
 [ 0.33306596 -0.4497311   0.65844177 ... -0.1667848  -2.14088161
  -0.22367863]
 [ 0.98226792  0.57648104  0.87899976 ...  0.38557993  0.37012294
  -1.24481511]
 ...
 [-1.0239342  -1.34514829 -1.72477356 ...  1.72420224  0.40600813
  -1.67254315]
 [-1.1460063  -0.55585006 -0.06354943 ...  0.76673139 -0.64436658
  -0.31156717]
 [-0.89734092 -0.92808442 -0.97449704 ... -0.93360767 -1.04206242
   1.21609842]]
(6400, 207)


ValueError: operands could not be broadcast together with shapes (6400,207) (518,) 

In [ ]:
predictions

In [17]:
from sklearn.metrics import confusion_matrix
import itertools

def draw_cm(model, filename):
    y_predict = model.predict_classes(X_test)
    cm = confusion_matrix(y_test, y_predict)
    print(cm)
    
    df_cm = pd.DataFrame(cm, index = [i for i in genre_names],
                      columns = [i for i in genre_names])
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111)
    cax = ax.matshow(df_cm, cmap='hot')
    fig.colorbar(cax)
    ax.set_xticks(np.arange(8))
    ax.set_yticks(np.arange(8))
    ax.set_xticklabels(genre_names)
    ax.set_yticklabels(genre_names)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "white")

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(filename)
    plt.show()

draw_cm(model, 'DNN_cm.png')

ValueError: Mix of label input types (string and number)